# pytest私房手册

## Fixture

### Teardown和Cleanup

有两种方式可以执行`Teardown`操作，一种是使用`yield`，一种是为`request`添加`addfinalizer`，但是两者有细微的差别：
```python
import pytest


@pytest.fixture
def f1():
    print('fixture 1 start!')
    yield 42
    raise ValueError('something wrong happened!')
    print('do some clean work!')
    print('fixture 1 end!')


@pytest.fixture
def f2(request):
    print('fixture 2 start!')

    def clean_work():
        print('do some clean work!')
        print('fixture 2 end!')

    request.addfinalizer(clean_work)

    raise ValueError('something wrong happened!')
    return 42


def test_fixture_1(f1):
    print(f1)


def test_fixture_2(f2):
    print(f2)
```
对于`test_fixture_1`，抛出错误之后的代码不会执行。但是对`test_fixture_2`，只要`addfinalizer`被添加到`request`上，那么不管之后是不是有异常，回调函数都会执行。`request`是内置的`fixture`，可以理解为调用`fixture`的测试函数。

另外，如果一个测试函数包含多个包含`Teardown`代码的`fixture`，其中一个`fixture`抛出了异常，不会影响其它的`fixture`：
```python
import pytest


@pytest.fixture
def f1():
    print('fixture 1 start')
    yield 42
    print('fixture 1 end')


@pytest.fixture
def f2():
    print('fixture 2 start')
    raise ValueError('f2 error')
    yield 33
    print('fixture 2 end')


def test_func(f1, f2):
    print('test function start:', f1, f2)
```
`f2`这个`fixture`抛出了异常，测试函数不会执行，`f2`异常后的代码不会执行，但是`f1`会正常执行完毕，打印信息为：
```
fixture 1 start
fixture 2 start
fixture 1 end
```
所以，`fixture`因为仅只完成一个功能点，尽量为原子操作。

### fixture可用性

一个`fixture`是否可用，是看它的`scope`，但是是以测试函数的角度去看的，比如：
```python
import pytest


@pytest.fixture
def order():
    return []


@pytest.fixture
def outer(order, inner):
    order.append("outer")


class TestOne:
    @pytest.fixture
    def inner(self, order):
        order.append("one")

    def test_order(self, order, outer):
        assert order == ["one", "outer"]


class TestTwo:
    @pytest.fixture
    def inner(self, order):
        order.append("two")

    def test_order(self, order, outer):
        assert order == ["two", "outer"]
```
当调用`test_order`，会依次执行`order`和`outer`，执行`outer`时，接收`inner`，显然`inner`对于`outer`而言，是不可见的。但实际上，可见不可见是对测试函数而言的，对于`test_order`来说，`inner`是可见的，所以在执行`outer`这个`fixture`时，`inner`这个`fixture`会正常执行。

### fixture搜索顺序

在包结构中，fixture会一直向上搜索，只要文件夹包含`__init__.py`，pytest就会认为是测试路径的一部分，一直搜索到最后一个包含`__init__.py`的文件夹，注意几点：
1. 只会向上搜索，不会向下搜索。
2. 同一个目录内的文件夹和文件视为同级的范围
3. 一个目录下的conftest.py文件里的fixture对当前范围内的所有文件可见。

### fixture的初始化顺序

fixture的初始化顺序和它的定义位置，执行顺序什么的都没有关系。
1. 由它的范围决定，简单来说，session->package->module->class->function
2. 相同范围的fixture由依赖关系决定初始化顺序。
3. 自动运行的fixture在其范围内会首先运行。
4. 自动运行的fixture a请求的fixture b也会成为自动运行的，不过a被使用的时候才会这样。

### 通过request获取测试上下文

通过内置的fixture`request`可以获取测试上下文，比如：
- `request.module`获取测试函数所在模块：
    ```python
    import pytest
    import smtplib

    @pytest.fixture(scope="module")
    def smtp_connection(request):
        server = getattr(request.module, "smtpserver", "smtp.gmail.com")
        smtp_connection = smtplib.SMTP(server, 587, timeout=5)
        yield smtp_connection
        print("finalizing {} ({})".format(smtp_connection, server))
        smtp_connection.close()
    ```
    测试函数：
    ```python
    smtpserver = "mail.python.org"  # will be read by smtp fixture

    def test_showhelo(smtp_connection):
        assert 0, smtp_connection.helo()
    ```
- 获取marker传递的值，node不是很清楚作用，待补充：
    ```python
    import pytest

    @pytest.fixture
    def fixt(request):
        marker = request.node.get_closest_marker("fixt_data")
        if marker is None:
            # Handle missing marker in some way...
            data = None
        else:
            data = marker.args[0]

        # Do something with the data
        return data

    @pytest.mark.fixt_data(42)
    def test_fixt(fixt):
        assert fixt == 42
    ```

### fixture工厂

fixture可以返回一个函数，动态产出数据：
```python
@pytest.fixture
def make_customer_record():
    def _make_customer_record(name):
        return {"name": name, "orders": []}

    return _make_customer_record


def test_customer_records(make_customer_record):
    customer_1 = make_customer_record("Lisa")
    customer_2 = make_customer_record("Mike")
    customer_3 = make_customer_record("Meredith")
```

### 参数化fixture

fixture可以参数化：
```python
# content of conftest.py
import pytest
import smtplib


@pytest.fixture(scope="module", params=["smtp.gmail.com", "mail.python.org"])
def smtp_connection(request):
    smtp_connection = smtplib.SMTP(request.param, 587, timeout=5)
    yield smtp_connection
    print("finalizing {}".format(smtp_connection))
    smtp_connection.close()
```
调用`smtp_connection`的测试函数相当于会运行2次，参数可以通过`request.param`获取。

默认情况下，打印测试项的时候，会在测试函数后添加`param`的值以示区分，可以通过`ids`参数指定名称，`ids`还可以接受一个函数，动态定义名称：
```python
# content of test_ids.py
import pytest


@pytest.fixture(params=[0, 1], ids=["spam", "ham"])
def a(request):
    return request.param


def test_a(a):
    pass


def idfn(fixture_value):
    if fixture_value == 0:
        return "eggs"
    else:
        return None


@pytest.fixture(params=[0, 1], ids=idfn)
def b(request):
    return request.param


def test_b(b):
    pass
```
上面例子加`--collect-only`运行（表示只收集打印测试项，不运行），结果如下：
```
$ pytest --collect-only
=========================== test session starts ============================
platform linux -- Python 3.x.y, pytest-6.x.y, py-1.x.y, pluggy-1.x.y
cachedir: $PYTHON_PREFIX/.pytest_cache
rootdir: $REGENDOC_TMPDIR
collected 10 items

<Module test_anothersmtp.py>
  <Function test_showhelo[smtp.gmail.com]>
  <Function test_showhelo[mail.python.org]>
<Module test_ids.py>
  <Function test_a[spam]>
  <Function test_a[ham]>
  <Function test_b[eggs]>
  <Function test_b[1]>
<Module test_module.py>
  <Function test_ehlo[smtp.gmail.com]>
  <Function test_noop[smtp.gmail.com]>
  <Function test_ehlo[mail.python.org]>
  <Function test_noop[mail.python.org]>

======================= 10 tests collected in 0.12s ========================
```

### fixture参数中包含marks

`Pytest.param()`可用于在参数化fixture的值集中应用`marks`，其方式与`@pytest.mark.parameterize`使用`marks`的方式相同:
```python
import pytest


@pytest.fixture(params=[0, 1, pytest.param(2, marks=pytest.mark.skip)])
def data_set(request):
    return request.param


def test_data(data_set):
    pass
```
运行的时候可以看到，参数2被忽略：
```
$ pytest test_fixture_marks.py -v
=========================== test session starts ============================
platform linux -- Python 3.x.y, pytest-6.x.y, py-1.x.y, pluggy-1.x.y -- $PYTHON_PREFIX/bin/python
cachedir: $PYTHON_PREFIX/.pytest_cache
rootdir: $REGENDOC_TMPDIR
collecting ... collected 3 items

test_fixture_marks.py::test_data[0] PASSED                           [ 33%]
test_fixture_marks.py::test_data[1] PASSED                           [ 66%]
test_fixture_marks.py::test_data[2] SKIPPED (unconditional skip)     [100%]

======================= 2 passed, 1 skipped in 0.12s =======================
```

### 根据fixture实例自动分组测试

Pytest在测试运行期间最小化活动fixture的数量。如果有一个参数化的fixture，所有的测试会先用第一个参数创建一个fixture实例，运行使用了该实例的所有测试项，然后终结这个fixture，再用第二个参数创建第二个fixture实例，运行与该实例有关的测试项。相当于根据fixture的参数对所有测试项进行了分组。
```python
import pytest


@pytest.fixture(scope="module", params=["mod1", "mod2"])
def modarg(request):
    param = request.param
    print("  SETUP modarg", param)
    yield param
    print("  TEARDOWN modarg", param)


@pytest.fixture(scope="function", params=[1, 2])
def otherarg(request):
    param = request.param
    print("  SETUP otherarg", param)
    yield param
    print("  TEARDOWN otherarg", param)


def test_0(otherarg):
    print("  RUN test0 with otherarg", otherarg)


def test_1(modarg):
    print("  RUN test1 with modarg", modarg)


def test_2(otherarg, modarg):
    print("  RUN test2 with otherarg {} and modarg {}".format(otherarg, modarg))
```
仔细观察输出：
```
$ pytest -v -s test_module.py
=========================== test session starts ============================
platform linux -- Python 3.x.y, pytest-6.x.y, py-1.x.y, pluggy-1.x.y -- $PYTHON_PREFIX/bin/python
cachedir: $PYTHON_PREFIX/.pytest_cache
rootdir: $REGENDOC_TMPDIR
collecting ... collected 8 items

test_module.py::test_0[1]   SETUP otherarg 1  # otherarg的scope是函数，因此每次执行测试函数都会执行otherarg fixture
  RUN test0 with otherarg 1
PASSED  TEARDOWN otherarg 1

test_module.py::test_0[2]   SETUP otherarg 2
  RUN test0 with otherarg 2
PASSED  TEARDOWN otherarg 2

# modarg的scope是模块，测试才开始创建
test_module.py::test_1[mod1]   SETUP modarg mod1  
  RUN test1 with modarg mod1
PASSED

# 参数为mod1的modarg这个fixture还被test2使用，所以先执行test2，而不是继续执行下一个test_1
test_module.py::test_2[mod1-1]   SETUP otherarg 1  
  RUN test2 with otherarg 1 and modarg mod1
PASSED  TEARDOWN otherarg 1

test_module.py::test_2[mod1-2]   SETUP otherarg 2
  RUN test2 with otherarg 2 and modarg mod1
PASSED  TEARDOWN otherarg 2

test_module.py::test_1[mod2]   TEARDOWN modarg mod1
  SETUP modarg mod2
  RUN test1 with modarg mod2
PASSED
test_module.py::test_2[mod2-1]   SETUP otherarg 1
  RUN test2 with otherarg 1 and modarg mod2
PASSED  TEARDOWN otherarg 1

test_module.py::test_2[mod2-2]   SETUP otherarg 2
  RUN test2 with otherarg 2 and modarg mod2
PASSED  TEARDOWN otherarg 2
  TEARDOWN modarg mod2


============================ 8 passed in 0.12s =============================
```
可见，上面的测试函数执行的顺序为0->0->1->2->2->1->2->2，而不是0->0->1->1->2->2。modarg是模块范围，2个参数，通过调整执行顺序，这个fixture一共执行了2次。而不是test1执行2次，test2执行2次。

之前这一点理解有误，以为如果fixture是模块范围，则会在脚本运行最初就执行一次fixture，结束才调用finalizer终止。实际是引用它的测试函数执行才会创建这个fixture，然后与这个fixture有关的其它的测试函数依次运行，最后调用finalizer终止这个fixture。

### 通过`usefixtures`在类和模块中使用fixture

有时候fixture不需要当作参数传递给测试方法，此时在类或者模块中，我们可以使用`pytest.mark.usefixtures`来指定所需的fixture，而不需要每个测试方法都把`fixture`当成参数进行传递。
```python
import os
import shutil
import tempfile

import pytest


@pytest.fixture
def cleandir():
    old_cwd = os.getcwd()
    newpath = tempfile.mkdtemp()
    os.chdir(newpath)
    yield
    os.chdir(old_cwd)
    shutil.rmtree(newpath)
```
可以直接装饰一个类，则每个测试方法都会执行该fixture。
```python
# content of test_setenv.py
import os
import pytest


@pytest.mark.usefixtures("cleandir")
class TestDirectoryInit:
    def test_cwd_starts_empty(self):
        assert os.listdir(os.getcwd()) == []
        with open("myfile", "w") as f:
            f.write("hello")

    def test_cwd_again_starts_empty(self):
        assert os.listdir(os.getcwd()) == []
```
在模块范围内，可以直接该语句：
```python
pytestmark = pytest.mark.usefixtures("cleandir")
```
设置，可以在配置文件`.ini`中使用，此时所有测试用例都会执行该fixture，而不需要将这个fixture当成参数传递给测试方法才会执行fixture：
```python
# content of pytest.ini
[pytest]
usefixtures = cleandir
```

### 覆盖不同级别的fixture

当fixture和本地参数同名时，本地参数会覆盖掉fixture:
```python
tests/
    __init__.py

    conftest.py
        # content of tests/conftest.py
        import pytest

        @pytest.fixture
        def username():
            return 'username'

        @pytest.fixture
        def other_username(username):
            return 'other-' + username

    test_something.py
        # content of tests/test_something.py
        import pytest
        
        # 当参数和fixture同名时，优先参数
        @pytest.mark.parametrize('username', ['directly-overridden-username'])
        def test_username(username):
            assert username == 'directly-overridden-username'
        
        # 注意，注意，other_username是fixture，此时它的参数本地参数username而不是fixture username
        # 正常情况下，定义了本地参数，则一定要传入测试函数，此时相当于通过fixture传入，不会报错
        @pytest.mark.parametrize('username', ['directly-overridden-username-other'])
        def test_username_other(other_username):
            assert other_username == 'other-directly-overridden-username-other'
```

可以通过设置`indirect`参数改变这种行为，此时使用fixture，忽略本地参数：
```python
import pytest


@pytest.fixture()
def username():
    return 'shl'


@pytest.mark.parametrize('username', ['telecomshy'], indirect=['username'])
def test_username(username):
    assert username == 'shl'
```

如果使用了`request.param`，情况又不一样了。我们在fixture参数化的时候展示过，当fixture包含`params`参数时，`request.param`总是`fixture`的参数：
```python
@pytest.fixture(params=['shl'])
def f(request):
    return request.param


def test_f(f):
    assert f == 'shl'
```
即使测试函数包含本地参数，`request.param`仍然是fixture的参数：
```python
@pytest.fixture(params=['shl'])
def f(request):
    return request.param


@pytest.mark.parametrize('q', ['shy'])
def test_f(q, f):
    assert f == 'shl'
```
但是当测试函数的本地参数和fixture同名，且本地参数包含`indirect`关键字时，`request.param`却变成了本地参数...
```python
@pytest.fixture(params=['shl'])
def f(request):
    return request.param


@pytest.mark.parametrize('f', ['shy'], indirect=True)
def test_f(f):
    assert f == 'shy'
```

### 使用其它项目的fixture

pytest以项目为入口点，一般只能看到本项目内confest.py文件定义的fixture。如果想要看到别的项目的fixture，假设在`mylibrary.fixtures`中有一些`fixture`，现在想重用它们到`app/tests`目录。可以在`app/tests/conftest.py`中定义指向该模块的`pytest_plugins`:
```python
pytest_plugins = "mylibrary.fixtures"
```

In [21]:
import requests

headers = {
    'Connection': 'keep-alive',
    'Content-Length': 389,
    'User-Agent':
    'Mozilla/5.0 (Linux; Android 12; RMX2071 Build/RKQ1.211103.002; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/86.0.4240.99 XWEB/3211 MMWEBSDK/20220303 Mobile Safari/537.36 MMWEBID/8674 MicroMessenger/8.0.21.2120(0x28001557) Process/appbrand0 WeChat/arm64 Weixin NetType/WIFI Language/zh_CN ABI/arm64 miniProgram/wx1e03b862a87656cd',
    'Content-Type': 'application/json',
    'Accept': '*/*',
    'Origin': 'https://whtgzh.whcst.com',
    'X-Requested-With': 'com.tencent.mm',
    'Sec-Fetch-Site': 'same-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://whtgzh.whcst.com/apps/whtgzh/web/index.html',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
}

url = 'https://wht-online.whcst.com/einvoicefront/einvoice/cardRechargeDtlForEInvoice/V1'

# 修改jsoninstssn和reqtime
data = {
    "joininstid": "00000000",
    "joininstssn": "202204282155023564",
    "reqdate": "20220428",
    "reqtime": "215502",
    "reqchanneltype": "00",
    "termid": "10000032",
    "mchntid": "000000010000001",
    "termseq": "000000010000001",
    "oprid": "000000010000001",
    "sign": "2c38e7478d16f278988368230f64df5564fa42bf",
    "data": {
        "cardno": "8898130344",
        "currpage": "1",
        "memopenid": "20200407142540816UDAQJ0JE5OQ56Z5",
        "recnumperpage": "100"
    }
}

result = requests.post(url=url, json=data)

result.text

'{"totalpage":1.0,"cardrechargelist":[{"orderid":"000611494051","txntype":14,"txntime":"2022-04-24 16:57:40.0","txnamt":10000,"carddptamt":0},{"orderid":"000610392160","txntype":14,"txntime":"2022-04-17 20:42:44.0","txnamt":10000,"carddptamt":0},{"orderid":"000608784679","txntype":14,"txntime":"2022-04-08 18:01:46.0","txnamt":10000,"carddptamt":0},{"orderid":"000606229135","txntype":14,"txntime":"2022-03-29 22:31:09.0","txnamt":10000,"carddptamt":0},{"orderid":"000606229126","txntype":14,"txntime":"2022-03-29 22:30:57.0","txnamt":3000,"carddptamt":0},{"orderid":"000604535621","txntype":14,"txntime":"2022-03-23 07:35:43.0","txnamt":10000,"carddptamt":0},{"orderid":"000602949447","txntype":14,"txntime":"2022-03-15 18:43:05.0","txnamt":10000,"carddptamt":0},{"orderid":"000601477577","txntype":14,"txntime":"2022-03-08 15:39:32.0","txnamt":10000,"carddptamt":0},{"orderid":"000600132224","txntype":14,"txntime":"2022-03-01 07:38:49.0","txnamt":10000,"carddptamt":0},{"orderid":"000598451680","